In [51]:
import gzip
from collections import defaultdict
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('reading data...')
data = list(readGz('train.json.gz'))
print('done')

reading data...
done


In [18]:
#problem 5
known_user_tr = [d['userID'] for d in data[:int(len(simplify_data)/2)]]
known_user_tr = list(set(known_user_tr))

In [76]:
simplify_data = [(d['userID'],d['businessID'],d['rating']) for d in data]
training_data = simplify_data[:int(len(simplify_data)/2)]
validation_data = simplify_data[int(len(simplify_data)/2):]

allRatings = []
userRating = defaultdict(list)
for i in training_data:
    allRatings.append(i[2])
    userRating[i[0]].append(i[2])
globalAverage = sum(allRatings)/len(allRatings)
userAverage = {}
for u in userRating:
    userAverage[u] = sum(userRating[u])/len(userRating[u])
MSE = 0
for d in validation_data:
    if d[0] in known_user_tr:
        MSE += (d[2]-userAverage[d[0]])**2
    else:
        MSE += (d[2]-globalAverage)**2
print('the globalaverage is ',globalAverage)
print('the MSE is ',MSE/len(validation_data))

the globalaverage is  4.18703
the MSE is  0.7163571192872857


In [142]:
b_allRating = []
bRating = defaultdict(list)
for i in training_data:
    b_allRating.append(i[2])
    bRating[i[1]].append(i[2])
b_globalAverage = sum(b_allRating)/len(b_allRating)
b_average = {}
for b in bRating:
    b_average[b] = sum(bRating[b])/len(bRating[b])
print('global busines rating is ',b_globalAverage)

global busines rating is  4.18703


In [158]:
#problem 6
from random import randint
simplify_data = [(d['userID'],d['businessID'],d['rating']) for d in data]
training_data = simplify_data[:int(len(simplify_data)/2)]
user_iden = [d[0] for d in training_data]
user_iden = list(set(user_iden))
business_iden = [d[1] for d in training_data]
business_iden = list(set(business_iden))
u_b_r = defaultdict(float)
user_business = defaultdict(list)
business_user = defaultdict(list)
for (u,b,r) in training_data:
    u_b_r[(u,b)] = r
    user_business[u].append(b)
    business_user[b].append(u)
for u in user_business:
    user_business[u] = list(set(user_business[u]))
for b in business_user:
    business_user[b] = list(set(business_user[b]))
beta_user = defaultdict(float)
beta_business = defaultdict(float)
alpha = randint(30,50)/10    
for u in user_iden:
    beta_user[u] = randint(-20,20)/100
for b in business_iden:
    beta_business[b] = randint(-20,20)/100
lam =1

    
    
    

In [168]:
#---------update------------------- 
SE_old = 0
SE_new = 3
while(abs(SE_new-SE_old)>0.001):
    sum_alpha = 0
    for d in training_data:
        sum_alpha += u_b_r[(d[0],d[1])]-(beta_user[d[0]]+beta_business[d[1]])
    alpha = sum_alpha/len(training_data)
    
    for u in user_iden:
        update_beta_user = 0
        for b in user_business[u]:
            update_beta_user += (u_b_r[(u,b)]-(alpha+beta_business[b]))/(lam+len(user_business[u]))
        beta_user[u] = update_beta_user
    
    for b in business_iden:
        update_beta_business = 0
        for u in business_user[b]:
            update_beta_business += (u_b_r[(u,b)]-(alpha+beta_user[u]))/(lam+len(business_user[b]))
        beta_business[b] = update_beta_business
    SE = 0
    for i in training_data:
        SE +=(alpha+beta_user[i[0]]+beta_business[i[1]]-u_b_r[(i[0],i[1])])**2
    #print( ' SE = ',SE)
    SE_old = SE_new
    SE_new = SE
    
    

In [160]:
MSE = 0
for (u,b,r) in validation_data:
    if u in user_iden:
        if b in business_iden:
            MSE += (alpha+beta_user[u]+beta_business[b]-u_b_r[(u,b)])**2
        else:
            MSE += (alpha+beta_user[u]-userAverage[u])**2
    else:
        if b in business_iden:
            MSE += (alpha+beta_business[b]-b_average[b])**2
        else:
            MSE += (alpha-globalAverage)**2
print('MSE for validation set is: ',MSE/len(validation_data))  
    

MSE for validation set is:  16.899844257640158


In [161]:
#problem7 
from operator import itemgetter
b_beta_user=sorted(beta_user.items(),key=itemgetter(1),reverse = True)
print('userID with largest value of beta is ',b_beta_user[0])
s_beta_user=sorted(beta_user.items(),key=itemgetter(1))
print('userID with smallest value of beta is ',s_beta_user[0])
b_beta_business = sorted(beta_business.items(),key = itemgetter(1), reverse = True)
print('businessID with largest value of beta is ',b_beta_business[0])
s_beta_business = sorted(beta_business.items(),key = itemgetter(1))
print('businessID with smallest value of beta is',s_beta_business[0])

userID with largest value of beta is  ('U357799541', 1.163195073560459)
userID with smallest value of beta is  ('U417838537', -2.8340258769043882)
businessID with largest value of beta is  ('B093985406', 1.169733638163109)
businessID with smallest value of beta is ('B241777680', -2.2339837734013415)


In [149]:
#####problem 8
from random import randint
simplify_data = [(d['userID'],d['businessID'],d['rating']) for d in data]
training_data = simplify_data[:int(len(simplify_data)/2)]
user_iden = [d[0] for d in training_data]
user_iden = list(set(user_iden))
business_iden = [d[1] for d in training_data]
business_iden = list(set(business_iden))
u_b_r = defaultdict(float)
user_business = defaultdict(list)
business_user = defaultdict(list)
for (u,b,r) in training_data:
    u_b_r[(u,b)] = r
    user_business[u].append(b)
    business_user[b].append(u)
for u in user_business:
    user_business[u] = list(set(user_business[u]))
for b in business_user:
    business_user[b] = list(set(business_user[b]))
beta_user = defaultdict(float)
beta_business = defaultdict(float)



In [166]:
alpha = randint(30,50)/10    
for u in user_iden:
    beta_user[u] = randint(-20,20)/100
for b in business_iden:
    beta_business[b] = randint(-20,20)/100
lam =7
#---------update------------------- 
SE_old = 0
SE_new = 3
while(abs(SE_new-SE_old)>0.001):
    sum_alpha = 0
    for d in training_data:
        sum_alpha += u_b_r[(d[0],d[1])]-(beta_user[d[0]]+beta_business[d[1]])
    alpha = sum_alpha/len(training_data)
    
    for u in user_iden:
        update_beta_user = 0
        for b in user_business[u]:
            update_beta_user += (u_b_r[(u,b)]-(alpha+beta_business[b]))/(lam+len(user_business[u]))
        beta_user[u] = update_beta_user
    
    for b in business_iden:
        update_beta_business = 0
        for u in business_user[b]:
            update_beta_business += (u_b_r[(u,b)]-(alpha+beta_user[u]))/(lam+len(business_user[b]))
        beta_business[b] = update_beta_business
    SE = 0
    for i in training_data:
        SE +=(alpha+beta_user[i[0]]+beta_business[i[1]]-u_b_r[(i[0],i[1])])**2
    
    SE_old = SE_new
    SE_new = SE
MSE = 0
for (u,b,r) in validation_data:
    if u in user_iden:
        if b in business_iden:
            MSE += (alpha+beta_user[u]+beta_business[b]-u_b_r[(u,b)])**2
        else:
            MSE += (alpha+beta_user[u]-userAverage[u])**2
    else:
        if b in business_iden:
            MSE += (alpha+beta_business[b]-b_average[b])**2
        else:
            MSE += (alpha-globalAverage)**2
print('MSE for validation set is: ',MSE/len(validation_data))  

MSE for validation set is:  16.7210142504965


In [167]:
predictions = open('prediction_Rating_q8.txt','w')
for l in open('pairs_Rating.txt'):
    if l.startswith('userID'):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in user_iden:
        if b in business_iden:
            rating = alpha+beta_user[u]+beta_user[b]
            predictions.write(u+'-'+i+','+str(rating)+'\n')
        else:
            rating = alpha+beta_user[u]
            predictions.write(u+'-'+i+','+str(rating)+'\n')
    else:
        if b in business_iden:
            rating = alpha+beta_business[b]
            predictions.write(u+'-'+i+','+str(rating)+'\n')
        else:
            rating = alpha
            predictions.write(u+'-'+i+','+str(rating)+'\n')
predictions.close()
        

In [ ]:
lambda = 2, MSE for validation set is:  16.828786437541
3   MSE for validation set is:  16.78888991765602
4 MSE for validation set is:  16.76329734951937
5    16.745203724742005
7 MSE for validation set is:  16.7210142504965

In [ ]:
MSE = 0
for (u,b,r) in validation_data:
    if u in user_iden:
        if b in business_iden:
            MSE += (alpha+beta_user[u]+beta_business[b]-u_b_r[(u,b)])**2
        else:
            MSE += (alpha+beta_user[u]-userAverage[u])**2
    else:
        if b in business_iden:
            MSE += (alpha+beta_business[b]-b_average[b])**2
        else:
            MSE += (alpha-globalAverage)**2
print('MSE for validation set is: ',MSE/len(validation_data))  
    

In [ ]:
#MSE on validation set
MSE = 0
for (u,b,r) in validation_data:
    if u in user_iden and b in business_iden:
        MSE += (alpha+beta_user[u]+beta_business[b]-u_b_r[(u,b)])**2
    if u in user_iden and b not in business_iden:
        MSE += (alpha+beta_user[u]-userAverage[u])**2
    if u not in user_iden and b in business_iden:
        MSE += (alpha+beta_business[b]-globalAverage)**2
    if u not in user_iden and b not in business_iden:
        MSE += (alpha-globalAverage)**2
print('MSE for validation set is: ',MSE/len(validation_data))  
    

In [87]:
sum_alpha = 0
for d in training_data:
    sum_alpha += u_b_r[(d[0],d[1])]-(beta_user[d[0]]+beta_business[d[1]])
alpha = sum_alpha/len(training_data)
print(alpha)


for u in user_iden:
    update_beta_user = 0
    for b in user_business[u]:
        update_beta_user += (u_b_r[(u,b)]-(alpha+beta_business[b]))/(lam+len(user_business[u]))
    beta_user[u] = update_beta_user
    

for b in business_iden:
    update_beta_business = 0
    for u in business_user[b]:
        update_beta_business += (u_b_r[(u,b)]-(alpha+beta_user[u]))/(lam+len(business_user[b]))
    beta_business[b] = update_beta_business

SE = 0
for i in training_data:
    SE +=(alpha+beta_user[i[0]]+beta_business[i[1]]-u_b_r[(i[0],i[1])])**2
print('MSE = ',SE/len(training_data))

4.223856747195829
MSE =  0.330111174768646
